# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install python-dotenv


In [2]:
import os
print(os.getenv("OPENAI_API_KEY"))
print(os.getenv("OPENAI_PROJECT_ID"))
import openai
print(openai.__version__)


None
None
1.97.0


In [3]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [18]:
!pip show langchain

Name: langchain
Version: 0.3.26
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /opt/anaconda3/envs/langchain-lab/lib/python3.11/site-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community


In [5]:
!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [8]:
pip install langchain==0.3.26 langchain-core==0.3.75 langchain-community==0.3.27 langchain-openai==0.3.28 pydantic==2.11.7 openai==1.97.0 huggingface-hub==0.33.4

Note: you may need to restart the kernel to use updated packages.


In [9]:
!pip install langchain_community


In [6]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [7]:
from openai import OpenAI
from langchain.prompts import ChatPromptTemplate

client = OpenAI()

In [8]:
#Replace None by your own value and justify
llm = ChatOpenAI(
    temperature=0.4,

)
# Neutrals values not allucinate but do not bring creativity.


In [9]:
prompt = ChatPromptTemplate.from_template(
    "Describe the following product in a clear and engaging way:\n\nProduct: {product}"
)


In [10]:
chain = LLMChain(llm=llm, prompt=prompt)


/var/folders/jk/_xp09qcs57sdz1r74tjmbdpr0000gn/T/ipykernel_5332/1929250370.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [11]:
product = "wireless noise-cancelling headphones"


In [12]:
chain.run(product)


/var/folders/jk/_xp09qcs57sdz1r74tjmbdpr0000gn/T/ipykernel_5332/1124456322.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


"Introducing the ultimate in audio technology - wireless noise-cancelling headphones! Immerse yourself in your favorite music or podcasts without any distractions from the outside world. With advanced noise-cancelling technology, these headphones block out unwanted sounds so you can focus on what matters most. The wireless design allows for freedom of movement, whether you're working out at the gym or commuting to work. Say goodbye to tangled cords and hello to crystal-clear sound quality. Upgrade your listening experience with these sleek and stylish headphones today!"

## SimpleSequentialChain

In [13]:
from langchain.chains import SimpleSequentialChain

In [14]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Write a short description for the following product:\n\n{input}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [15]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Create a catchy marketing slogan based on the following description:\n\n{input}"
)

# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [16]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [17]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Enjoy superior sound quality and uninterrupted music with these wireless noise-cancelling headphones. With advanced technology to eliminate background noise, these headphones provide a premium listening experience wherever you go. Comfortable and convenient, they are perfect for travel, work, or leisure.
"Block out the noise, elevate your sound with wireless noise-cancelling headphones!"

> Finished chain.


'"Block out the noise, elevate your sound with wireless noise-cancelling headphones!"'

**Repeat the above twice for different products**

In [18]:
product = "smart fitness watch"
overall_simple_chain.run(product)




> Entering new SimpleSequentialChain chain...
The smart fitness watch is a sleek and stylish wearable device that helps you track your daily activity levels, monitor your heart rate, and analyze your workouts. With features like GPS tracking, sleep monitoring, and notifications for calls and messages, this watch is the perfect companion for anyone looking to stay on top of their health and fitness goals.
"Stay stylish, stay fit with our smart fitness watch - your ultimate health companion!"

> Finished chain.


'"Stay stylish, stay fit with our smart fitness watch - your ultimate health companion!"'

## SequentialChain

In [26]:
from langchain.chains import SequentialChain

In [27]:
llm = ChatOpenAI(temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review into English:\n\n{review}"
)

chain_one = LLMChain(
    llm=llm,
    prompt=first_prompt,
    output_key="english_review"
)


In [28]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review in one sentence:\n\n{english_review}"
)

chain_two = LLMChain(
    llm=llm,
    prompt=second_prompt,
    output_key="summary"
)


In [29]:
# Prompt 3: translate English review to another language
third_prompt = ChatPromptTemplate.from_template(
    "Translate the following review into {language}:\n\n{english_review}"
)

chain_three = LLMChain(
    llm=llm,
    prompt=third_prompt,
    output_key="translated_review"
)


In [30]:
# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
    "Based on the following summary and translated review, write a polite follow-up message to the customer.\n\n"
    "Summary:\n{summary}\n\n"
    "Translated Review:\n{translated_review}"
)

chain_four = LLMChain(
    llm=llm,
    prompt=fourth_prompt,
    output_key="followup_message"
)


In [24]:
from langchain.chains import SequentialChain


In [31]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review", "language"],
    output_variables=[
        "english_review",
        "summary",
        "translated_review",
        "followup_message"
    ],
    verbose=True
)


In [33]:
overall_chain({
    "review": df.Review[5],
    "language": "German"
})




> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'language': 'German',
 'english_review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer is disappointed with the taste of the product, finding it mediocre and questioning whether it may be an old batch or counterfeit.',
 'translated_review': 'Ich finde den Geschmack mittelmäßig. Der Schaum hält nicht, das ist komisch. Ich kaufe die gleichen in Läden und der Geschmack ist viel besser... Alte Charge oder Fälschung!?',
 'followup_message': 'Dear valued customer,\n\nThank you for sharing your feedback with us. We are sorry to hear that you found the taste of our product to be mediocre and that the foam did not hold up as expected. We assure you that our products are always fresh and 

In [35]:
overall_chain({
    "review": df.Review[5],
    "language": "Italian"
})




> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'language': 'Italian',
 'english_review': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer is disappointed in the taste and quality of the product, suspecting it may be an old batch or counterfeit.',
 'translated_review': 'Trovo il gusto mediocre. La schiuma non tiene, è strano. Compro gli stessi in negozio e il sapore è molto migliore... Vecchio lotto o contraffatto!?',
 'followup_message': 'Dear valued customer,\n\nWe are sorry to hear that you were disappointed with the taste and quality of our product. We take your feedback seriously and would like to assure you that our products are not old batches or counterfeit. \n\nWe would appreciate the opportunity to learn more about y

In [36]:
overall_chain({
    "review": df.Review[5],
    "language": "Spanish"
})




> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'language': 'Spanish',
 'english_review': "I find the taste mediocre. The foam does not hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer is disappointed with the taste and foam quality of the product, suspecting that it may be an old batch or counterfeit.',
 'translated_review': 'Encuentro el sabor mediocre. La espuma no se mantiene, es extraño. Compro los mismos en tiendas y el sabor es mucho mejor... ¿Lote viejo o falsificado?',
 'followup_message': 'Dear valued customer,\n\nThank you for sharing your feedback with us. We are sorry to hear that you found the taste and foam quality of our product disappointing. We take these concerns seriously and would like to assure you that our products are always fresh and authentic.\n

**Repeat the above twice for different products or reviews**

## Router Chain

In [37]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [39]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [40]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [41]:
llm = ChatOpenAI(temperature=0)

In [42]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [43]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [44]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [45]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [47]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [48]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body is characterized by a continuous spectrum of wavelengths and intensities, which depends only on the temperature of the body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body is proportional to the frequency of the radiation and the temperature of the body."

In [49]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [50]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This genetic information is crucial for maintaining the integrity and stability of our cells and ultimately our entire body. \n\nAdditionally, DNA is constantly being replicated and repaired in our cells to ensure that genetic information is passed on accurately to the next generation of cells. This process is essential for the growth, development, and overall health of our bodies. \n\nIn summary, every cell in our body contains DNA because it is essential for the proper functioning and maintenance of a

In [51]:
chain.run("What is the difference between a stack and a queue in computer science?")




> Entering new MultiPromptChain chain...
computer science: {'input': 'What is the difference between a stack and a queue in computer science?'}
> Finished chain.


'In computer science, a stack and a queue are both data structures used to store and retrieve data, but they have different characteristics and behaviors.\n\nA stack is a data structure that follows the Last In, First Out (LIFO) principle, meaning that the last element added to the stack is the first one to be removed. Elements are added and removed from the top of the stack, which is also known as the "top of the stack." Stacks are typically used for tasks that require backtracking or undo operations, such as in function calls or expression evaluation.\n\nOn the other hand, a queue is a data structure that follows the First In, First Out (FIFO) principle, meaning that the first element added to the queue is the first one to be removed. Elements are added to the back of the queue and removed from the front, which is also known as the "front of the queue." Queues are commonly used in scenarios where data needs to be processed in the order it was received, such as in task scheduling or m

In [52]:
chain.run("What were the main causes of the French Revolution?")




> Entering new MultiPromptChain chain...
History: {'input': 'What were the main causes of the French Revolution?'}
> Finished chain.


"The main causes of the French Revolution can be attributed to a combination of social, political, and economic factors. \n\n1. Social Inequality: The French society was divided into three estates, with the clergy and nobility enjoying privileges and exemptions from taxes, while the common people, or the Third Estate, bore the burden of heavy taxation and poverty. This social inequality created resentment and discontent among the lower classes.\n\n2. Economic Crisis: France was facing a severe economic crisis due to years of extravagant spending by the monarchy, costly wars, and poor harvests. This led to widespread poverty, hunger, and unemployment, further fueling the discontent among the people.\n\n3. Political Corruption: The absolute monarchy of Louis XVI was characterized by corruption, inefficiency, and a lack of accountability. The monarchy's refusal to reform and address the grievances of the people only exacerbated the situation.\n\n4. Enlightenment Ideas: The Enlightenment i

**Repeat the above at least once for different inputs and chains executions - Be creative!**